# Oral Insulin Phase II Clinical Trial Data


This project wrangles the datasets offered by [Udacity](https://www.udacity.com/?utm_source=gsem_brand&utm_medium=ads_r&utm_campaign=12907726228_c&utm_term=128479270744&utm_keyword=udacity_e&gclid=Cj0KCQjwkbuKBhDRARIsAALysV6inYic9AGOkLLqOjh__ssJ6wn2Sv8GMedrW8NbIFiUpLtv-5aCt9QaAga7EALw_wcB) for an imaginary phase II clinical trial data, testing the efficacy, dose response, and common short term side effects called adverse reactions, for a new oral insulin product called Auralin and an injectiable insulin called Novodra, with 175 patients for each drug.

In [512]:
import pandas as pd
import re
import numpy as np

In [513]:
patients = pd.read_csv('patients.csv')
treatments = pd.read_csv('treatments.csv')
treatments_cut = pd.read_csv('treatments_cut.csv')
adverse_reactions = pd.read_csv('adverse_reactions.csv')

## Assess

In [514]:
patients

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,499,male,Mustafa,Lindström,2530 Victoria Court,Milton Mills,ME,3852.0,United States,207-477-0579MustafaLindstrom@jourrapide.com,4/10/1959,181.1,72,24.6
499,500,male,Ruman,Bisliev,494 Clarksburg Park Road,Sedona,AZ,86341.0,United States,928-284-4492RumanBisliev@gustr.com,3/26/1948,239.6,70,34.4
500,501,female,Jinke,de Keizer,649 Nutter Street,Overland Park,MO,64110.0,United States,816-223-6007JinkedeKeizer@teleworm.us,1/13/1971,171.2,67,26.8
501,502,female,Chidalu,Onyekaozulu,3652 Boone Crockett Lane,Seattle,WA,98109.0,United States,ChidaluOnyekaozulu@jourrapide.com1 360 443 2060,2/13/1952,176.9,67,27.7


In [515]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   patient_id    503 non-null    int64  
 1   assigned_sex  503 non-null    object 
 2   given_name    503 non-null    object 
 3   surname       503 non-null    object 
 4   address       491 non-null    object 
 5   city          491 non-null    object 
 6   state         491 non-null    object 
 7   zip_code      491 non-null    float64
 8   country       491 non-null    object 
 9   contact       491 non-null    object 
 10  birthdate     503 non-null    object 
 11  weight        503 non-null    float64
 12  height        503 non-null    int64  
 13  bmi           503 non-null    float64
dtypes: float64(3), int64(2), object(9)
memory usage: 55.1+ KB


In [516]:
patients


,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,499,male,Mustafa,Lindström,2530 Victoria Court,Milton Mills,ME,3852.0,United States,207-477-0579MustafaLindstrom@jourrapide.com,4/10/1959,181.1,72,24.6
499,500,male,Ruman,Bisliev,494 Clarksburg Park Road,Sedona,AZ,86341.0,United States,928-284-4492RumanBisliev@gustr.com,3/26/1948,239.6,70,34.4
500,501,female,Jinke,de Keizer,649 Nutter Street,Overland Park,MO,64110.0,United States,816-223-6007JinkedeKeizer@teleworm.us,1/13/1971,171.2,67,26.8
501,502,female,Chidalu,Onyekaozulu,3652 Boone Crockett Lane,Seattle,WA,98109.0,United States,ChidaluOnyekaozulu@jourrapide.com1 360 443 2060,2/13/1952,176.9,67,27.7


In [517]:
patients[patients['address'].isnull()]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
209,210,female,Lalita,Eldarkhanov,NaN,NaN,NaN,NaN,NaN,NaN,8/14/1950,143.4,62,26.2
219,220,male,Mỹ,Quynh,NaN,NaN,NaN,NaN,NaN,NaN,4/9/1978,237.8,69,35.1
230,231,female,Elisabeth,Knudsen,NaN,NaN,NaN,NaN,NaN,NaN,9/23/1976,165.9,63,29.4
234,235,female,Martina,Tománková,NaN,NaN,NaN,NaN,NaN,NaN,4/7/1936,199.5,65,33.2
242,243,male,John,O'Brian,NaN,NaN,NaN,NaN,NaN,NaN,2/25/1957,205.3,74,26.4
249,250,male,Benjamin,Mehler,NaN,NaN,NaN,NaN,NaN,NaN,10/30/1951,146.5,69,21.6
257,258,male,Jin,Kung,NaN,NaN,NaN,NaN,NaN,NaN,5/17/1995,231.7,69,34.2
264,265,female,Wafiyyah,Asfour,NaN,NaN,NaN,NaN,NaN,NaN,11/3/1989,158.6,63,28.1
269,270,female,Flavia,Fiorentino,NaN,NaN,NaN,NaN,NaN,NaN,10/9/1937,175.2,61,33.1
278,279,female,Generosa,Cabán,NaN,NaN,NaN,NaN,NaN,NaN,12/16/1962,124.3,69,18.4


In [518]:
patients.describe()

,patient_id,zip_code,weight,height,bmi
count,503.000000,491.000000,503.000000,503.000000,503.000000
mean,252.000000,49084.118126,173.434990,66.634195,27.483897
std,145.347859,30265.807442,33.916741,4.411297,5.276438
min,1.000000,1002.000000,48.800000,27.000000,17.100000
25%,126.500000,21920.500000,149.300000,63.000000,23.300000
50%,252.000000,48057.000000,175.300000,67.000000,27.200000
75%,377.500000,75679.000000,199.500000,70.000000,31.750000
max,503.000000,99701.000000,255.900000,79.000000,37.700000


In [519]:
patients.surname.value_counts()

Doe         6
Taylor      3
Jakobsen    3
Tucker      2
Aranda      2
           ..
Petersen    1
Mayberry    1
Baum        1
Vũ          1
Wubbels     1
Name: surname, Length: 466, dtype: int64

In [520]:
patients.address.value_counts()

123 Main Street        6
2778 North Avenue      2
2476 Fulton Street     2
648 Old Dear Lane      2
4262 Heron Way         1
                      ..
669 Archwood Avenue    1
1033 Wayside Lane      1
1753 Sherman Street    1
4438 Reeves Street     1
2403 Ripple Street     1
Name: address, Length: 483, dtype: int64

In [521]:
patients[patients.address.duplicated()]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
29,30,male,Jake,Jakobsen,648 Old Dear Lane,Port Jervis,New York,12771.0,United States,JakobCJakobsen@einrot.com+1 (845) 858-7707,8/1/1985,155.8,67,24.4
219,220,male,Mỹ,Quynh,NaN,NaN,NaN,NaN,NaN,NaN,4/9/1978,237.8,69,35.1
229,230,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
230,231,female,Elisabeth,Knudsen,NaN,NaN,NaN,NaN,NaN,NaN,9/23/1976,165.9,63,29.4
234,235,female,Martina,Tománková,NaN,NaN,NaN,NaN,NaN,NaN,4/7/1936,199.5,65,33.2
237,238,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
242,243,male,John,O'Brian,NaN,NaN,NaN,NaN,NaN,NaN,2/25/1957,205.3,74,26.4
244,245,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
249,250,male,Benjamin,Mehler,NaN,NaN,NaN,NaN,NaN,NaN,10/30/1951,146.5,69,21.6
251,252,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4


In [522]:
patients.weight.sort_values()

210     48.8
459    102.1
335    102.7
74     103.2
317    106.0
       ...  
144    244.9
61     244.9
283    245.5
118    254.5
485    255.9
Name: weight, Length: 503, dtype: float64

In [523]:
treatments

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.97
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32
...,...,...,...,...,...,...,...
275,albina,zetticci,45u - 51u,-,7.93,7.73,0.20
276,john,teichelmann,-,49u - 49u,7.90,7.58,NaN
277,mathea,lillebø,23u - 36u,-,9.04,8.67,0.37
278,vallie,prince,31u - 38u,-,7.64,7.28,0.36


In [524]:
treatments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   given_name    280 non-null    object 
 1   surname       280 non-null    object 
 2   auralin       280 non-null    object 
 3   novodra       280 non-null    object 
 4   hba1c_start   280 non-null    float64
 5   hba1c_end     280 non-null    float64
 6   hba1c_change  171 non-null    float64
dtypes: float64(3), object(4)
memory usage: 15.4+ KB


In [525]:
treatments.describe()

,hba1c_start,hba1c_end,hba1c_change
count,280.000000,280.000000,171.000000
mean,7.985929,7.589286,0.546023
std,0.568638,0.569672,0.279555
min,7.500000,7.010000,0.200000
25%,7.660000,7.270000,0.340000
50%,7.800000,7.420000,0.380000
75%,7.970000,7.570000,0.920000
max,9.950000,9.580000,0.990000


In [ ]:
sum(treatments.novodra.isnull())

In [ ]:
sum(treatments.auralin.isnull())

In [ ]:
sum(treatments.auralin.isnull())

In [ ]:
sum(treatments.novodra.isnull())

In [ ]:
treatments_cut

In [ ]:
treatments_cut

In [ ]:
treatments_cut.info()

In [ ]:
treatments_cut.describe()

#### Quality Issues
##### `patients` table

- Zip code has four digits sometimes
- Tim Neudorf height is 27 in
- Full state names sometimes, abbreviations other times
- Dsvid Gustafsson instead of David Gustafsson
- Missing demographic information (address - contact columns) 
- Erroneous datatypes (assigned sex, state, zip_code, and birthdate columns)
- Multiple phone number formats
- Default John Doe data
- Multiple records for Jakobsen, Gersten, Taylor
- kgs instead of lbs for Zaitseva weight

##### `treatments` table
- Missing HbA1c changes
- The letter 'u' in starting and ending doses for Auralin and Novodra
- Lowercase given names and surnames
- The record are in two tables instead of one (treatments and treatments_cut)
- Inaccurate HbA1c changes (leading 4s mistaken as 9s)
- Nulls represented as dashes (-) in auralin and novodra columns

##### `adverse_reactions` table
- Lowercase given names and surnames

#### Tidiness
- Contact column in `patients` table should be split into phone number and email
- Three variables in two columns in `treatments` table (treatment, start dose and end dose)
- Adverse reaction should be part of the `treatments` table
- Given name and surname columns in `patients` table duplicated in `treatments` and `adverse_reactions` tables

## Clean

In [ ]:
patients_clean = patients.copy()
treatments_clean = treatments.copy()
adverse_reactions_clean = adverse_reactions.copy()

In [ ]:
# missing_patients_info = patients_clean[patients_clean.address.isnull()]
# patients_clean.dropna(inplace=True)

In [ ]:

# patients_clean['zip_code']= patients_clean['zip_code'].astype(int)
# patients_clean['zip_code']= patients_clean['zip_code'].astype(int).astype(str)

# for i in range(len(patients_clean)):
#   if  (len(patients_clean['zip_code'].iloc[i]) == 4):
#    patients_clean['zip_code'].iloc[i] = '0' + patients_clean['zip_code'].iloc[i]


### Missing Data

#### `treatments`: Data is in two dataframes instead of one.

##### Define
Contact both treatments files into one file.

##### Code

In [ ]:
treatments_clean=pd.concat([treatments_clean, treatments_cut], ignore_index=True)

##### Test

In [ ]:
len(treatments_clean)

In [ ]:
treatments_clean.head()

#### `treatments`: Missing HbA1c changes and Inaccurate HbA1c changes (leading 4s mistaken as 9s)


##### Define
Recalculate the hba1c_end between hba1c_start and hba1c_end.

##### Code

In [ ]:
treatments_clean['hba1c_change'] = treatments_clean['hba1c_start'] - treatments_clean['hba1c_end']

##### Test

In [ ]:
treatments_clean.describe() 

In [ ]:
treatments_clean.head()

### Tidiness

#### Contact column in `patients` table contains two variables: phone number and email

Use regular expressions to seperate the emails and phone number into two different variables

##### Code

In [ ]:
patients_clean["emails"]= patients_clean.contact.str.extract(r'([a-zA-Z]+@\w+\.[a-z]+)')
patients_clean["phone_number"]= patients_clean.contact.str.extract(r'([^a-zA-Z][0-9]+.+[0-9]+[-0-9]+[^a-zA-Z])')
patients_clean.drop('contact',axis=1,inplace=True)

##### Test

In [ ]:
patients_clean.head()

In [ ]:
patients.info()

#### Three variables in two columns in `treatments` table (treatment, start dose and end dose)

##### Define
Split the auralin and novodra coloumn into three columns: treatment, start dose, end dose.

##### Code

In [ ]:
tratments_aur_null= treatments_clean.auralin!= '-'
tratments_nov_null= treatments_clean.novodra!= '-'
treatments_clean['treatment'] = 'auralin'
treatments_clean.treatment[tratments_nov_null]=  'novodra'

In [ ]:
treatments_clean.info()

In [ ]:
treatments_clean.dropna(inplace = True)
aur= treatments_clean["auralin"].str.split(" - ", n=0, expand = True)
aur=aur[aur[0].iloc[:] != '-']
aur_first = aur[0]
aur_last=  aur[1]

In [ ]:
treatments_clean.dropna(inplace = True)
nov= treatments_clean["novodra"].str.split(" - ", n=0, expand = True)
nov=nov[nov[0].iloc[:] != '-']
nov_first = nov[0]
nov_last=  nov[1]

In [ ]:
treatments_clean['start dose'] =aur_first
treatments_clean['start dose'].fillna(nov_first,inplace =True)
treatments_clean.head()

In [ ]:
treatments_clean['end dose'] =aur_last
treatments_clean['end dose'].fillna(nov_last,inplace =True)
treatments_clean.drop(['auralin','novodra'],axis=1,inplace=True)
treatments_clean.head()

##### Test

In [ ]:
treatments_clean.head()

In [ ]:
treatments_clean.info()

#### Adverse reaction should be part of the `treatments` table

##### Define

Join the treatments and adverse_reactions_clean tables

##### Code

In [ ]:
treatments_clean2=pd.merge(treatments_clean, adverse_reactions_clean, on=['surname','given_name'], how='outer')

##### Test

In [ ]:
treatments_clean2.head()

In [ ]:
treatments_clean2.head()

In [ ]:
adverse_reactions_clean.head()

#### Given name and surname columns in `patients` table duplicated in `treatments` and `adverse_reactions` tables  and Lowercase given names and surnames

##### Define
Replace the given_name and surname in treatments table with the patient id using the patients_clean table

##### Code

In [ ]:
treatments_clean2.head()

In [ ]:
treatments_clean2.given_namename=treatments_clean2.given_name.map(lambda i: i[0].upper() + i[1:] )
treatments_clean2.surname=treatments_clean2.surname.map(lambda i: i[0].upper() + i[1:] )

In [ ]:
patients_id_surname= patients_clean[['patient_id','surname','given_name']]
treatments_clean3=pd.merge(patients_id_surname, treatments_clean2, on=['surname','given_name'], how='inner')

In [ ]:
treatments_clean3.drop(['given_name','surname'],axis=1,inplace=True)

##### Test

In [ ]:
treatments_clean3.head(8)

In [ ]:
treatments_clean3.info()

### Quality

#### Zip code is a float not a string and Zip code has four digits sometimes

##### Define
Change the data type to be a string, and add a zero to the beginning of the four digits zip codes


##### Code

In [ ]:
missing_patients_info = patients_clean[patients_clean.address.isnull()]
patients_clean.dropna(inplace=True)


In [ ]:
patients_clean['zip_code']= patients_clean['zip_code'].astype(int)
patients_clean['zip_code']= patients_clean['zip_code'].astype(int).astype(str)

for i in range(len(patients_clean)):
  if  (len(patients_clean['zip_code'].iloc[i]) == 4):
   patients_clean['zip_code'].iloc[i] = '0' + patients_clean['zip_code'].iloc[i]


##### Test

In [ ]:
# Your testing code here
patients_clean.info()

#### Tim Neudorf height is 27 in instead of 72 in

##### Define
Change his height to 72

##### Code

In [ ]:
patients_clean.height.loc[patients_clean.height<50] = 72

##### Test

In [ ]:
patients_clean[patients_clean.surname=='Neudorf']

In [ ]:
sum(patients_clean.height<50)

#### Full state names sometimes, abbreviations other times

##### Define
Write a function to change all the states names to be abberviated 

##### Code

In [ ]:
def abbreviate(state):


    if (len(state)>2):
        state=state.strip()
        if (" " in state):
            state_split = state.split(" ")
            state=(state_split[0][0]+state_split[1][0]).upper()
        else:
             state=(state[0]+ state[1]).upper()
        
    return state
        
        

for i in range(len(patients_clean)):
    state_=patients_clean.state.iloc[i]
    abbreviated=abbreviate(state_)
    patients_clean.replace({'state':{patients_clean.state.iloc[i]:abbreviated}},inplace=True)
    


##### Test

In [ ]:
patients_clean.sample(10)

In [ ]:
patients_clean.state.value_counts()

#### Dsvid Gustafsson

##### Define
Correct the name to be David

##### Code

In [ ]:
patients_clean.head()

In [ ]:
patients_clean.given_name.loc[patients_clean.given_name=='Dsvid'] = 'David'

##### Test

In [ ]:
patients_clean[patients_clean.given_name=='Dsvid'] 

#### Erroneous datatypes (assigned sex, birthdate, and treatment columns) and The letter 'u' in starting and ending doses columns for Auralin and Novodra

##### Define
Change the columns types and remove the 'u' letter

##### Code

In [ ]:
patients_clean['assigned_sex']=patients_clean['assigned_sex'].astype('category')
patients_clean['birthdate']=patients_clean['birthdate'].astype('datetime64')
treatments_clean3['treatment']=treatments_clean3['treatment'].astype('category')

In [ ]:
treatments_clean3['start dose']=treatments_clean3['start dose'].map(lambda i: i[0:-1] )
treatments_clean3['end dose']=treatments_clean3['end dose'].map(lambda i: i[0:-1] )

##### Test

In [ ]:
treatments_clean3.info()

In [ ]:
patients_clean.info()

#### The phone numbers have different formats

##### Define
Change all the numbers to have xxx-xxx-xxxx format

##### Code

In [ ]:
for index, row in patients_clean.iterrows():
    if ('+1' in row['phone_number']):
        row['phone_number']=row['phone_number'].replace('+1','')        
    patients_clean.phone_number[index]=re.sub('\D', '', row['phone_number'])
    patients_clean.phone_number[index]=patients_clean.phone_number[index][0:3]+'-'+patients_clean.phone_number[index][3:6]+'-'+ patients_clean.phone_number[index][6:]

##### Test

In [ ]:
patients_clean.tail()

In [ ]:
patients_clean.info()

#### Default John Doe data

##### Define
Drop all the rows with the default John Doe data

##### Code

In [ ]:
patients_clean.drop(patients_clean[patients_clean.surname=='Doe'].index,axis=0,inplace=True)

##### Test

In [ ]:
len(patients_clean)

In [ ]:
patients_clean[patients_clean.surname=='Doe'].index

#### Multiple records for Jakobsen, Gersten, Taylor

##### Define
*Your definition here.*

##### Code

In [ ]:
 patients_clean.drop_duplicates(subset=['surname', 'emails','address'], keep='last',inplace=True)

##### Test

In [ ]:
patients_clean[patients_clean.surname=='Jakobsen'] 

In [ ]:
patients_clean[patients_clean.surname=='Gersten']

In [ ]:
patients_clean[patients_clean.surname=='Taylor'] 

In [ ]:
len(patients_clean)

#### kgs instead of lbs for Zaitseva weight

##### Define
*Mutltiply the weight by 2.205 to convert it to lbs*


##### Code

In [528]:
ind=patients_clean[patients_clean.surname=='Zaitseva'].index
patients_clean.weight[ind]=patients_clean.weight[ind]*2.205

<ipython-input-528-9752b5bce17b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patients_clean.weight[ind]=patients_clean.weight[ind]*2.205


##### Test

In [529]:
patients_clean[patients_clean.surname=='Zaitseva'].weight

210    107.604
Name: weight, dtype: float64